### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [5]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.large-data :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [6]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio （Go back here everytime you want to restart.）

In [27]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 1000);

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 50 cross below the MA 200 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [28]:
(def MA50-vec-aapl [])
(def MA200-vec-aapl [])
(def MA50-vec-f [])
(def MA200-vec-f [])
(while (not= (get-date) "2000-12-26")
    (do
      (def tics (deref available-tics)) ;20 ms
      (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl))
      (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl))
      (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f ))
      (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
        (if (> MA50 MA200)
          (order "AAPL" 1 :print false) 
          (order "AAPL" 0 :remaining true))))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
        (if (> MA50 MA200)
          (order "F" 1  :print false) 
          (order "F" 0 :remaining true )))
      (update-eval-report (get-date))
      (next-date))
(end-order)

nil

In [12]:
;(pprint/print-table (take 200 (deref portfolio-value)))
(count (deref order-record))

2439

### Check portfolio record

In [13]:
;; view final portfolio
(view-portfolio)


| :asset | :price | :aprc | :quantity | :tot-val |
|--------+--------+-------+-----------+----------|
|   cash |    N/A |   N/A |       N/A |    15499 |


nil

In [26]:
;; view portfolio value and return
(view-portfolio-record 10)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage |
|------------+------------+------------+----------+-------+-----------|
| 1980-12-15 |      $1000 |      0.00% |    0.00% | $0.00 |     0.00% |
| 1981-02-27 |      $1000 |      0.00% |    0.00% | $0.00 |     0.00% |
| 1981-03-02 |      $1001 |      0.00% |    0.05% | $0.00 |     0.00% |
| 1981-03-03 |      $1001 |      0.00% |    0.06% | $0.00 |     0.00% |
| 1981-03-04 |      $1001 |      0.00% |    0.05% | $0.00 |     0.00% |
| 1981-03-05 |       $999 |      0.00% |   -0.01% | $0.00 |     0.00% |
| 1981-03-06 |      $1010 |      0.00% |    0.45% | $0.00 |     0.00% |
| 1981-03-09 |      $1022 |      0.00% |    0.97% | $0.00 |     0.00% |
| 1981-03-10 |      $1007 |      0.00% |    0.34% | $0.00 |     0.00% |
| 1981-03-11 |      $1006 |      0.00% |    0.30% | $0.00 |     0.00% |


nil

### Generate evaluation report

In [25]:
(eval-report 434)
;; (update-eval-report)
;; (deref eval-report-data)


|      :date | :pnl-pt |      :sharpe | :tot-val |                                                                                                                                                                                                                                                                    :vol |
|------------+---------+--------------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1981-02-26 |      $0 |        0.00% |    $1000 |                                                                                                                                                                                                                                                                   0.00% |
| 1981-02-27 |      $0 |     9459.33% |    $1001

| 1981-04-01 |      $0 |    -5852.08% |     $985 |                                                                                                                                                                                                                                                                   0.00% |
| 1981-04-02 |     $-1 |    -5683.89% |     $951 |                                                                                                                                                                                                                                                                   0.00% |
| 1981-04-03 |     $-2 |    -9319.48% |     $923 |                                                                                                                                                                                                                                                                   0.00% |
| 1981-04-06 |     $-1 |    -6930.23% |     $945 

| 1981-05-08 |     $-1 |         NaN% |     $902 |                                                                                                                                                                                                                                                                    NaN% |
| 1981-05-11 |      $0 |         NaN% |     $944 |                                                                                                                                                                                                                                                                    NaN% |
| 1981-05-12 |      $0 |         NaN% |     $958 |                                                                                                                                                                                                                                                                    NaN% |
| 1981-05-13 |      $0 |         NaN% |    $1029 

| 1981-06-16 |      $2 |         NaN% |    $1228 |                                                                                                                                                                                                                                                                    NaN% |
| 1981-06-17 |      $1 |         NaN% |    $1189 |                                                                                                                                                                                                                                                                    NaN% |
| 1981-06-18 |      $1 |         NaN% |    $1145 |                                                                                                                                                                                                                                                                    NaN% |
| 1981-06-19 |      $1 |         NaN% |    $1100 

| 1981-07-23 |      $0 |         NaN% |     $906 |                                                                                                                                                                                                                                                                    NaN% |
| 1981-07-24 |      $0 |         NaN% |     $904 |                                                                                                                                                                                                                                                                    NaN% |
| 1981-07-27 |     $-1 |         NaN% |     $806 |                                                                                                                                                                                                                                                                    NaN% |
| 1981-07-28 |     $-1 |         NaN% |     $743 

...

| 1982-06-25 |     $-1 |         NaN% |     $543 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-06-28 |     $-1 |         NaN% |     $573 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-06-29 |     $-1 |         NaN% |     $573 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-06-30 |     $-2 |         NaN% |     $479 

| 1982-08-03 |     $-2 |         NaN% |     $357 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-08-04 |     $-2 |         NaN% |     $243 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-08-05 |     $-2 |         NaN% |     $243 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-08-06 |     $-2 |         NaN% |     $220 

| 1982-09-09 |      $0 |         NaN% |    $1056 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-09-10 |      $0 |         NaN% |    $1244 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-09-13 |      $1 |         NaN% |    $1378 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-09-14 |      $1 |         NaN% |    $1351 

| 1982-10-15 |      $3 |         NaN% |    $2174 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-10-18 |      $3 |         NaN% |    $2083 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-10-19 |      $4 |         NaN% |    $2334 |                                                                                                                                                                                                                                                                    NaN% |
| 1982-10-20 |      $3 |         NaN% |    $2182 

nil

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)